In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import glob
import os

from sklearn import datasets, linear_model, metrics

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

## Data Preparation

In [114]:
def read_inputs():
    holidays = pd.read_csv("../data/holidays_events.csv")
    holidays['date'] = pd.to_datetime(holidays['date'])
    holidays = holidays[holidays["transferred"]==False]
    holidays = holidays[holidays["type"].isin(['Holiday','Additional','Bridge','Event','Transfer'])]
    holidays['holiday_flag'] = 1
    
    oil = pd.read_csv("../data/oil.csv")
    oil["date"] = pd.to_datetime(oil['date'])
    oil_new = pd.DataFrame({"date":list(pd.date_range(start=oil.date.min(),end=oil.date.max()))})
    oil_new = pd.merge(oil_new,oil,how="left",on=["date"])
    oil_new['dcoilwtico'] = oil_new['dcoilwtico'].ffill()
    oil_new['dcoilwtico'] = oil_new['dcoilwtico'].bfill()

    stores = pd.read_csv("../data/stores.csv")
    transaction = pd.read_csv("../data/transactions.csv")

    sample = pd.read_csv("../data/sample_submission.csv")

    train = pd.read_csv("../data/train.csv")
    train['date'] = pd.to_datetime(train['date'])

    test = pd.read_csv("../data/test.csv")
    test['date'] = pd.to_datetime(test['date'])

    return holidays, oil_new, stores, transaction, train, test, sample


In [155]:
def create_train_data(train,test,holidays,oil,stores,transaction):

    # Filter out 0 sales
    train = train[train['sales']>0]

    # Get holiday flag
    train = pd.merge(train,holidays[['date','holiday_flag']],how='left',on=['date'])
    train['holiday_flag'].fillna(0,inplace=True)
    test = pd.merge(test,holidays[['date','holiday_flag']],how='left',on=['date'])
    test['holiday_flag'].fillna(0,inplace=True)

    # Get oil prices
    train = pd.merge(train,oil[['date','dcoilwtico']],how='left',on=['date'])
    test = pd.merge(test,oil[['date','dcoilwtico']],how='left',on=['date'])

    # Get store cluster
    train = pd.merge(train,stores[['store_nbr','cluster']],how='left',on=['store_nbr'])
    test = pd.merge(test,stores[['store_nbr','cluster']],how='left',on=['store_nbr'])

    # Get transaction

    # Get seasonality
    
    dummy = pd.concat([train[['date']], test[['date']]])
    dummy = dummy.drop_duplicates(subset=['date'])
    dummy = pd.concat([dummy, pd.get_dummies(dummy['date'].dt.month,prefix='month',drop_first=True)], axis = 1)
    train = pd.merge(train, dummy, how='left',on =['date'])
    test = pd.merge(test, dummy, how='left',on =['date'])
    del dummy

    return train, test


## Exploratory Analysis

In [60]:
# Gettting average plots per cluster to check time series
df_train = pd.merge(train,stores[["store_nbr","cluster"]],how="left",on=["store_nbr"])
df_train_gpy = df_train.groupby(["date","cluster"]).agg({"sales":['sum','mean',"count"]}).reset_index()
df_train_gpy.columns = [' '.join(col).strip() for col in df_train_gpy.columns.values]
df_train_gpy = df_train_gpy[df_train_gpy['sales sum']>0]
df_train_gpy.head(2)
df_train_gpy.to_excel("../output/sales_plot.xlsx",index=False)

## Model building

In [156]:
holidays, oil_new, stores, transaction, train, test, sample = read_inputs()
train, test = create_train_data(train, test, holidays, oil, stores, transaction)

In [162]:
def scale_variables(X_train,y_train,X_test):
    scale_x = StandardScaler()
    scale_x.fit(X_train)
    X_train = scale_x.transform(X_train)
    X_test = scale_x.transform(X_test)

    scale_y = StandardScaler()
    scale_y.fit(y_train)
    y_train = scale_y.transform(y_train)

    return X_train, y_train, X_test, scale_x, scale_y

def inverse_scale_variables(X_train, y_train, X_test, train_pred, test_pred, scale_x, scale_y):
    train_pred = scale_y.inverse_transform(train_pred)
    test_pred = scale_y.inverse_transform(test_pred)
    y_train = scale_y.inverse_transform(y_train)
    X_train = scale_x.inverse_transform(X_train)
    X_test = scale_x.inverse_transform(X_test)

    return X_train, y_train, X_test, train_pred, test_pred

def build_model(X_train, y_train, model_name = "linear_regression"):

    if model_name == "linear_regression":
        # create linear regression object
        reg = linear_model.LinearRegression()        
        # train the model using the training sets
        reg.fit(X_train, y_train)
        return reg
    

    

In [127]:
features = ['onpromotion', 'holiday_flag','dcoilwtico', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11','month_12']
dv = ['sales']

In [173]:
for i in test['store_nbr'].tolist()[0:1]:
    for j in test['family'].tolist()[0:1]:
        df_train = train[(train['store_nbr']==i) & (train['family']==j)][features+['date']+dv].copy()
        df_test = test[(test['store_nbr']==i) & (test['family']==j)][features+['date']].copy()
        df_train.set_index(['date'],inplace=True)
        df_test.set_index(['date'],inplace=True)
        X_train = df_train[features]
        y_train = df_train[dv]
        X_test = df_test[features]

        X_train_sc, y_train_sc, X_test_sc, scale_x, scale_y = scale_variables(X_train, y_train, X_test)

        model_obj = build_model(X_train_sc, y_train_sc, model_name = "linear_regression")
        train_pred = model_obj.predict(X_train_sc)
        test_pred = model_obj.predict(X_test_sc)

        X_train_sc, y_train_sc, X_test_sc, train_pred, test_pred = inverse_scale_variables(X_train_sc, y_train_sc, X_test_sc, train_pred, test_pred, scale_x, scale_y)
        train_temp = pd.concat([X_train,y_train],axis=1)
        train_temp['train_pred'] = train_pred
        test_temp = X_test.copy()
        test_temp['test_pred'] = test_pred
        

        

In [174]:

print("Train MAPE : ",mean_absolute_percentage_error(y_train_sc, train_pred))
# print("Test MAPE : ",mean_absolute_percentage_error(y_train, train_pred))



Train MAPE :  0.8168067327233302


In [175]:
X_train.head(1)

,onpromotion,holiday_flag,dcoilwtico,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
date,,,,,,,,,,,,,,
2013-01-02,0,0.0,93.14,0,0,0,0,0,0,0,0,0,0,0


In [152]:
y_train

array([[4.4408921e-16],
       [2.0000000e+00],
       [3.0000000e+00],
       ...,
       [1.0000000e+00],
       [1.0000000e+00],
       [4.0000000e+00]])

In [154]:
y_train

,sales
date,
2013-01-01,0.0
2013-01-02,2.0
2013-01-03,3.0
2013-01-04,3.0
2013-01-05,5.0
...,...
2017-08-11,1.0
2017-08-12,6.0
2017-08-13,1.0
